# WGAN-GP 二次元图片生成演示

本笔记本演示如何使用WGAN-GP模型进行二次元图片的生成和数据增强。

## 目录
1. 环境准备
2. 数据加载
3. 模型训练
4. 图像生成
5. 结果可视化
6. 数据增强应用

## 1. 环境准备

In [ ]:
import sys
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

# 添加项目路径
project_root = os.path.abspath('..')
sys.path.append(project_root)

from models.wgan_gp import WGANGP
from src.utils import (
    load_config,
    set_seed,
    get_device,
    get_dataloader,
    save_image_grid,
    denormalize
)

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 加载配置和数据

In [ ]:
# 加载配置文件
config_path = '../config/config.yaml'
config = load_config(config_path)

# 设置随机种子
set_seed(config['device']['seed'])

# 获取设备
device = get_device(config)

print("配置加载完成！")
print(f"图像大小: {config['data']['image_size']}x{config['data']['image_size']}")
print(f"批次大小: {config['training']['batch_size']}")
print(f"潜在维度: {config['model']['latent_dim']}")

In [ ]:
# 加载数据集（需要先准备数据）
try:
    dataloader = get_dataloader(config, shuffle=True)
    print(f"数据集大小: {len(dataloader.dataset)}")
    print(f"批次数量: {len(dataloader)}")
    
    # 可视化一些真实图像
    real_batch = next(iter(dataloader))
    print(f"批次形状: {real_batch.shape}")
    
    # 显示样本
    fig, axes = plt.subplots(2, 8, figsize=(16, 4))
    for i in range(16):
        ax = axes[i // 8, i % 8]
        img = denormalize(real_batch[i]).permute(1, 2, 0).cpu().numpy()
        ax.imshow(img)
        ax.axis('off')
    plt.suptitle('真实图像样本')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"无法加载数据集: {e}")
    print("提示: 请将二次元图像数据放置在 data/processed/ 目录下")

## 3. 创建和训练模型

### 3.1 创建模型

In [ ]:
# 创建WGAN-GP模型
model = WGANGP(config, device)

print("模型创建成功！")
print(f"生成器参数量: {sum(p.numel() for p in model.generator.parameters()):,}")
print(f"判别器参数量: {sum(p.numel() for p in model.discriminator.parameters()):,}")

### 3.2 训练模型（简化版）

注意: 这里只演示几个epoch的训练。完整训练请使用 `src/train.py` 脚本。

In [ ]:
# 简短的训练演示
num_demo_epochs = 5

# 检查是否有数据
if 'dataloader' in locals():
    print(f"开始训练 {num_demo_epochs} 个epoch...")
    
    for epoch in range(num_demo_epochs):
        d_losses = []
        g_losses = []
        
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_demo_epochs}")
        
        for batch_idx, real_images in enumerate(pbar):
            real_images = real_images.to(device)
            
            # 训练判别器
            d_loss, wd = model.train_discriminator(real_images)
            d_losses.append(d_loss)
            
            # 训练生成器
            if batch_idx % config['training']['n_critic'] == 0:
                g_loss = model.train_generator()
                g_losses.append(g_loss)
            
            pbar.set_postfix({
                'D_loss': f'{np.mean(d_losses):.4f}',
                'G_loss': f'{np.mean(g_losses):.4f}' if g_losses else 'N/A'
            })
            
            # 只训练几个批次作为演示
            if batch_idx >= 10:
                break
        
        print(f"Epoch {epoch+1}: D_loss={np.mean(d_losses):.4f}, G_loss={np.mean(g_losses):.4f}")
    
    print("\n演示训练完成！")
    print("注意: 这只是简短的演示。完整训练需要运行200+ epochs。")
else:
    print("跳过训练演示 - 需要先准备数据集")

## 4. 加载预训练模型（如果有）

In [ ]:
# 如果有预训练模型，可以加载
checkpoint_path = '../checkpoints/saved_models/best_model.pth'

if os.path.exists(checkpoint_path):
    print(f"加载预训练模型: {checkpoint_path}")
    epoch = model.load_checkpoint(checkpoint_path)
    print(f"模型训练至 epoch {epoch}")
else:
    print("未找到预训练模型")
    print("使用随机初始化的模型（仅用于演示）")

## 5. 生成图像

In [ ]:
# 生成一些图像
num_images = 64
print(f"生成 {num_images} 张图像...")

model.generator.eval()
with torch.no_grad():
    fake_images = model.generate(num_images=num_images, batch_size=num_images)

print(f"生成完成！形状: {fake_images.shape}")

## 6. 可视化生成结果

In [ ]:
# 显示生成的图像网格
fig, axes = plt.subplots(8, 8, figsize=(16, 16))
axes = axes.flatten()

for i in range(min(64, num_images)):
    img = denormalize(fake_images[i]).permute(1, 2, 0).cpu().numpy()
    axes[i].imshow(img)
    axes[i].axis('off')

plt.suptitle('WGAN-GP 生成的二次元图像', fontsize=16)
plt.tight_layout()
plt.show()

## 7. 潜在空间探索

### 7.1 潜在空间插值

In [ ]:
# 在两个随机点之间插值
def interpolate_latent(model, z1, z2, num_steps=10):
    """在潜在空间中插值"""
    alphas = np.linspace(0, 1, num_steps)
    interpolated = []
    
    with torch.no_grad():
        for alpha in alphas:
            z = (1 - alpha) * z1 + alpha * z2
            img = model.generator(z)
            interpolated.append(img)
    
    return torch.cat(interpolated, dim=0)

# 生成两个随机潜在向量
z1 = torch.randn(1, config['model']['latent_dim'], device=device)
z2 = torch.randn(1, config['model']['latent_dim'], device=device)

# 执行插值
interpolated_images = interpolate_latent(model, z1, z2, num_steps=10)

# 显示插值结果
fig, axes = plt.subplots(1, 10, figsize=(20, 2))
for i in range(10):
    img = denormalize(interpolated_images[i]).permute(1, 2, 0).cpu().numpy()
    axes[i].imshow(img)
    axes[i].axis('off')
    axes[i].set_title(f't={i/9:.1f}')

plt.suptitle('潜在空间插值 (从左到右)', fontsize=14)
plt.tight_layout()
plt.show()

### 7.2 随机采样多样性

In [ ]:
# 从不同的随机种子生成
num_samples = 16
random_samples = []

with torch.no_grad():
    for i in range(num_samples):
        z = torch.randn(1, config['model']['latent_dim'], device=device)
        img = model.generator(z)
        random_samples.append(img)

random_samples = torch.cat(random_samples, dim=0)

# 显示随机样本
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.flatten()

for i in range(num_samples):
    img = denormalize(random_samples[i]).permute(1, 2, 0).cpu().numpy()
    axes[i].imshow(img)
    axes[i].axis('off')
    axes[i].set_title(f'Sample {i+1}')

plt.suptitle('随机采样展示多样性', fontsize=14)
plt.tight_layout()
plt.show()

## 8. 数据增强应用

### 8.1 批量生成用于数据增强

In [ ]:
# 生成大量图像用于数据增强
num_augment_images = 100
augment_dir = '../data/augmented/demo'
os.makedirs(augment_dir, exist_ok=True)

print(f"生成 {num_augment_images} 张图像用于数据增强...")

batch_size = 50
num_batches = (num_augment_images + batch_size - 1) // batch_size

image_count = 0
for batch_idx in tqdm(range(num_batches)):
    current_batch_size = min(batch_size, num_augment_images - batch_idx * batch_size)
    
    with torch.no_grad():
        z = torch.randn(current_batch_size, config['model']['latent_dim'], device=device)
        fake_images = model.generator(z)
    
    # 保存图像
    for i in range(current_batch_size):
        img = denormalize(fake_images[i]).permute(1, 2, 0).cpu().numpy()
        img = (img * 255).astype(np.uint8)
        Image.fromarray(img).save(f'{augment_dir}/generated_{image_count:06d}.png')
        image_count += 1

print(f"✓ 已生成 {image_count} 张图像")
print(f"保存位置: {augment_dir}")

### 8.2 数据增强效果评估

这里展示如何将生成的图像与原始数据集合并用于下游任务。

In [ ]:
print("数据增强流程：")
print("""\n1. 收集原始数据集 (例如: 1000张图像)
2. 训练WGAN-GP模型
3. 使用训练好的模型生成新图像 (例如: 5000张)
4. （可选）质量筛选：使用FID或人工筛选高质量图像
5. 合并原始数据和生成数据
6. 在扩充后的数据集上训练下游任务模型
7. 评估性能提升

预期效果：
- 分类准确率提升 2-5%
- 更好的泛化性能
- 减少过拟合
""")

## 9. 总结

本笔记本演示了：

1. ✓ 如何加载和配置WGAN-GP模型
2. ✓ 如何训练模型（简化演示）
3. ✓ 如何生成新的二次元图像
4. ✓ 如何在潜在空间中探索
5. ✓ 如何应用于数据增强

### 下一步

- 完整训练：使用 `python src/train.py` 训练完整模型
- 批量生成：使用 `python src/generate.py` 生成大量图像
- 质量评估：使用评估脚本计算FID和IS分数
- 实际应用：将生成的图像用于实际的数据增强任务

### 参考资料

- README.md: 完整的项目说明
- docs/技术文档.md: 详细的技术实现
- docs/项目报告.md: 项目总结报告